# Configuring your computer to use Python for scientific computing

*This document was generated from a Jupyter notebook, which you can download [here](configure_your_computer.ipynb).*

<hr>

## Downloading and installing Miniconda

If you already have Anaconda or Miniconda installed on your computed, you can skip to the next section.

I find it is best to install Miniconda from the command line. To do so, following the instructions on [the Miniconda installation webpage](https://docs.anaconda.com/miniconda/install/#quick-command-line-install). Be sure to select your operating system. The only exception is if you have an Intel Mac. In that case, do the following on the command line.

```
mkdir -p ~/miniconda3
curl https://repo.anaconda.com/miniconda/Miniconda3-latest-MacOSX-x86_64.sh -o ~/miniconda3/miniconda.sh
bash ~/miniconda3/miniconda.sh -b -u -p ~/miniconda3
rm ~/miniconda3/miniconda.sh
source ~/miniconda3/bin/activate
conda init --all
```

## The be25 environment

To set up the environment you need for this class, execute the following on the command line.

    conda env create -f https://be25.github.io/handouts/be25.yml

## Activating the environment

After the environment is created, you need to **activate it**. To do this from the command line, execute the following.

    conda activate be25

**You will need to activate the environment every time you open a new terminal (or PowerShell) window.** (You can have this happen automatically if you like by adding `conda activate be25` to your configuration file, e.g., `.bashrc`. I don't recommend this, since you will probably have more environments on your machine.)

## Launching JupyterLab

You can launch JupyterLab is from the command line. To launch JupyterLab, type the following on the command line (again, *after* you have done `conda activate be25`).

    jupyter lab
    
You can specify the browser you want using, for example,

    jupyter lab --browser=firefox

Within the JupyterLab window, you will have the option to launch a notebook, a console, a terminal, or a text editor.

## Checking your distribution

With JupyterLab running, use the JupyterLab launcher (you can get a new launcher by clicking on the `+` icon on the left pane of your JupyterLab window) to launch a notebook. In the first cell (the box next to the `[ ]:` prompt), paste the code below. To run the code, press `Shift+Enter` while the cursor is active inside the cell. It may take a few moments for the cell to run. When it's done, you should see a plot that looks like the one below. If you do, you have a functioning Python environment for scientific computing!

In [1]:
import numpy as np
import pandas as pd

import eqtk

import bokeh.io
import bokeh.plotting

bokeh.io.output_notebook()

# Chemical reactions for aspartic acid
rxns = """
     <=> OH⁻ + H⁺  ; 1e-14
H₃D⁺ <=> H₂D + H⁺  ; {Ka1}
H₂D  <=> HD⁻ + H⁺  ; {Ka2}
HD⁻  <=> D²⁻ + H⁺  ; {Ka3}
""".format(Ka1=10**(-1.99), Ka2=10**(-3.9), Ka3=10**(-10.002))

# Stoichiometric matrix
N = eqtk.parse_rxns(rxns)

# Initial concentrations
c0 = {
    "H⁺":   0,
    "OH⁻":  0,
    "H₃D⁺": 0,
    "H₂D":  0.001,
    "HD⁻":  0,
    "D²⁻":  0,
}

# Set up pH variable
fixed_c = pd.DataFrame(data=-np.ones((400, 6)), columns=c0.keys())
pH = np.linspace(0, 14, 400)
fixed_c["H⁺"] = 10**(-pH)

# Solve for equilibrium concentrations
c = eqtk.fixed_value_solve(c0=c0, fixed_c=fixed_c, N=N, units='M')

# Parse concentrations
c['pH'] = -np.log10(c['[H⁺] (M)'])
c['H₃D⁺'] = c['[H₃D⁺] (M)'] / c0["H₂D"]
c['H₂D'] = c['[H₂D] (M)'] / c0["H₂D"]
c['HD⁻'] = c['[HD⁻] (M)'] / c0["H₂D"]
c['D²⁻'] = c['[D²⁻] (M)'] / c0["H₂D"]

# Make the plot
p = bokeh.plotting.figure(
    height=250,
    width=420,
    y_axis_label="relative abundance",
    x_axis_label="pH",
    x_range=[0, 14],
)

p.line(c['pH'], c['H₃D⁺'], color='#4c78a8', line_width=2, legend_label="H₃D⁺")
p.line(c['pH'], c['H₂D'], color='#f58518', line_width=2, legend_label="H₂D")
p.line(c['pH'], c['HD⁻'], color='#e45756', line_width=2, legend_label="HD⁻")
p.line(c['pH'], c['D²⁻'], color='#72b7b2', line_width=2, legend_label="D²⁻")

p.legend.location = 'center_right'

bokeh.io.show(p)

Loading BokehJS ...

## Computing environment

In [2]:
%load_ext watermark
%watermark -v -p numpy,pandas,eqtk,bokeh,jupyterlab

Python implementation: CPython
Python version       : 3.12.8
IPython version      : 8.30.0

numpy     : 1.26.4
pandas    : 2.2.3
eqtk      : 0.1.3
bokeh     : 3.6.2
jupyterlab: 4.2.5

